In [5]:
import numpy as np
import matplotlib.pyplot as plt
import ipdb

np.random.seed(0)

In [6]:
def board_to_int(board):
    x = 0
    for i in range(9):
        x *= 3
        x += board.flat[8 - i]
    return x

def int_to_board(x):
    board = np.zeros((3, 3), 'int')
    for i in range(9):
        board.flat[i] = x % 3
        x //= 3
    return board

In [7]:
def test_coding_encoding(tests_cnt):
    for i in range(tests_cnt):
        d = np.random.randint(3, size=(3, 3))
        x = board_to_int(d)
        d2 = int_to_board(x)
        if not ((d2 == d).all()):
            print('err')
            return

test_coding_encoding(0)

In [8]:
def calc_allowed_actions(state):
    board = int_to_board(state)
    return np.flatnonzero(board == 0)

In [9]:
states_count = 3 ** 9
allowed_actions = list(map(calc_allowed_actions, range(states_count)))
base_states = np.repeat(-1, states_count)
transforms = np.empty(states_count, dtype=tuple)

In [39]:
def find_similar_states(base_state):
    board = int_to_board(base_state)
    for i in range(4):
        state = board_to_int(board)
        base_states[state] = base_state
        transforms[state] = np.array((i, 0))
        board_reversed = np.fliplr(board)
        reversed_state = board_to_int(board_reversed)
        transforms[reversed_state] = np.array((i, 1))
        base_states[reversed_state] = base_state
        board = np.rot90(board)
        
        
for state in range(states_count):
    if base_states[state] == -1:
        find_similar_states(state)

In [40]:
class Judge:
    def __init__(self):
        self.DRAW = 0
        self.PLAYING = 3
        self.results = list(map(self.calc_game_status, range(states_count)))

    def get_board_lines(self, state):
        board = int_to_board(state)
        board_rotated = np.rot90(board)
        diag1 = np.diag(board)
        diag2 = np.diag(board_rotated)
        return np.vstack([board, board_rotated, diag1, diag2])
        
    def line_winner(self, line):
        for player in range(1, 3):
            if (line == player).all():
                return player
        return 0

    def calc_game_status(self, state):
        all_lines = self.get_board_lines(state)
        for line in all_lines:
            if self.line_winner(line):
                return self.line_winner(line)
        return self.PLAYING if len(allowed_actions[state]) else self.DRAW
    
    def get_result(self, state):
        return self.results[state]

In [41]:
judge = Judge()

In [152]:
class AgentSARSA:
    def __init__(self, epsilon=0.01, gamma=0.5, alpha=0.1):
        self.epsilon = epsilon
        self.gamma = gamma
        #self.alpha = alpha
        self.q = np.zeros((states_count, 9))
        self.count = np.zeros((states_count, 9), 'int')
        self.prev_state = 0
        self.prev_action = 0
        
    def choose_action(self, state):
        q = self.q[state].take(allowed_actions[state])
        indices_of_best = np.flatnonzero(q == q.max())
        best_actions = allowed_actions[state].take(indices_of_best)
        good_choice = np.random.choice(best_actions)
        rand_choice = np.random.choice(allowed_actions[state])
        return np.random.choice([good_choice, rand_choice], p=[1 - self.epsilon, self.epsilon])
    
    def alpha(self, cnt):
        return 0.1
        return np.log(cnt + 1) ** -1
        
    def get_action(self, state):
        action = self.choose_action(state)
        self.count[state][action] += 1
        alpha = self.alpha(self.count[self.prev_state][self.prev_action] + 1)
        if state != -1:
            self.q[self.prev_state][self.prev_action] += alpha * (self.gamma * self.q[state][action]
                                                                  - self.q[self.prev_state][self.prev_action])
        self.prev_state = state
        self.prev_action = action
        return action
    
    def put_reward(self, reward):
        alpha = self.alpha(self.count[self.prev_state][self.prev_action] + 1)
        self.q[self.prev_state][self.prev_action] += alpha * (reward
                                                              - self.q[self.prev_state][self.prev_action])
        self.state = -1

In [167]:
class Simulator:
    def __init__(self, judge, agents):
        self.agents = agents
        self.judge = judge
        
    def play_games(self, games_cnt):
        for i in range(games_cnt):
            if i % 100 == 0 or i == games_cnt - 1:
                progress = round((i + 1) / games_cnt * 100, 1)
                print('\r' * 10 + str(progress) + '%', end='')
            state = 0
            player = 0
            while self.judge.get_result(state) == 3:
                action = self.agents[player].get_action(state)
                state += (player + 1) * 3 ** action
                player = (player + 1) % 2
                state = base_states[state]
                
            winner = self.judge.get_result(state)
            if winner:
                self.agents[winner - 1].put_reward(100)
                self.agents[1 - (winner - 1)].put_reward(-100)
            else:
                for agent in self.agents:
                    agent.put_reward(1)
        #print()

In [168]:
SARSA_agents = AgentSARSA(alpha=0.05), AgentSARSA(alpha=0.05)

sim = Simulator(judge, SARSA_agents)

In [187]:
SARSA_agents[1].epsolon = 0.01

import time
cl = time.clock()
sim.play_games(100000)
cl = time.clock() - cl
print('Duration: ', cl)

100.0%Duration:  48.84771599999999


In [170]:
class RandomAgent:
    def get_action(self, state):
        return np.random.choice(allowed_actions[state])

In [171]:
class StrategiesComparator:
    def __init__(self, agents, win_reward, draw_reward):
        self.agents = agents
        self.win_reward = win_reward
        self.draw_reward = draw_reward
        self.rewards = np.zeros(2)
        
    def compare(self, tests):
        for i in range(tests):
            state = 0
            player = 0
            while judge.get_result(state) == 3:
                action = self.agents[player].get_action(state)
                state += (player + 1) * 3 ** action
                player = (player + 1) % 2
                state = base_states[state]
                
            winner = judge.get_result(state)
            if winner:
                self.rewards[winner - 1] += self.win_reward
            else:
                self.rewards += self.draw_reward
        return self.rewards

In [197]:
rand_agent = RandomAgent()
ag = SARSA_agents[1]
ag.epsilon = 0
comp = StrategiesComparator([rand_agent, ag], 100, 100)

tests = 50000
rewards = comp.compare(tests)
r = rewards[1]
print(r / (100 * tests))

0.99834


In [198]:
comp.agents[1].epsilon

0

In [134]:
class PlayingInterface:
    def __init__(self, agents, role=1):
        self.judge = judge
        if role == 1:
            self.agent = agents[1]
        else:
            self.agent = agents[0]
        self.agent.epsilon = 0
        self.agent_role = 3 - role
        self.role = role        
        
    def start(self):
        self.agent.state = -1
        transform = []
        res = 3
        state = 0
        if self.role == 2:
            action = self.agent.get_action(state)
            state += self.agent_role * 3 ** action
        self.print_board(state)
        
        while res == 3:
            y, x = list(map(int, input().split()))
            action = y * 3 + x
            state += self.role * 3 ** action
            res = self.judge.get_result(state)
            
            transform = transforms[state]
            state = base_states[state]
            
            if res == 3:
                action = self.agent.get_action(state)
                state += self.agent_role * 3 ** action
                res = self.judge.get_result(state)
            
            board = int_to_board(state)
            board = np.rot90(board, transform[0])
            if transform[1]:
                board = np.fliplr(board)
            state = board_to_int(board)
            
            self.print_board(state)

        if res == self.role:
            print("You won")
        elif res == self.agent_role:
            print("You lost")
        elif res == 0:
            print("Draw")
        
        print('Restart? (y/n)')
        ans = input()
        if ans == 'y':
            self.start()
        
    def print_board(self, state):
        b = int_to_board(state)
        print(b)

In [149]:
p = PlayingInterface(SARSA_agents, 1)

In [150]:
p.start()

[[0 0 0]
 [0 0 0]
 [0 0 0]]
1 1
[[2 0 0]
 [0 1 0]
 [0 0 0]]
0 1
[[2 1 0]
 [0 1 0]
 [0 2 0]]
1 0
[[2 1 0]
 [1 1 2]
 [0 2 0]]
2 0
[[2 1 2]
 [1 1 2]
 [1 2 0]]
2 2
[[2 1 2]
 [1 1 2]
 [1 2 1]]
Draw
Restart? (y/n)
n


In [34]:
b = np.zeros((3, 3), 'int')
b[1][1] = 1
x = board_to_int(b)
print(x)
print(base_states[x])
print(int_to_board(x))
SARSA_agents[1].q[x]

81
81
[[0 0 0]
 [0 1 0]
 [0 0 0]]


array([-14.75368773, -21.61311116, -16.46562046, -22.75215832,
         0.        , -25.95705198, -18.0340858 , -20.60184509,   2.9912765 ])

# Q-learning

In [525]:
class AgentQ(AgentSARSA):
    def get_action(self, state):
        action = self.choose_action(state)
        q_max = self.q[state].take(allowed_actions[state]).max()
        
        self.q[self.prev_state][self.prev_action] += self.alpha * (self.gamma * q_max
                                                                   - self.q[self.prev_state][self.prev_action])
        self.prev_state = state
        self.prev_action = action
        return action

In [527]:
Q_agents = [AgentQ(alpha=0.05), AgentQ(alpha=0.05)]

sim2 = Simulator(judge, Q_agents)
sim2.play_games(100000)
print(1)

100.0%
1


In [528]:
p = PlayingInterface(Q_agents[1])

In [529]:
p.start()

[[0 0 0]
 [0 0 0]
 [0 0 0]]
1 1
[[0 0 2]
 [0 1 0]
 [0 0 0]]
0 1
[[0 1 2]
 [0 1 0]
 [0 2 0]]
2 2
[[2 1 2]
 [0 1 0]
 [0 2 1]]
1 0
[[2 1 2]
 [1 1 2]
 [0 2 1]]
2 0
Draw
